In [58]:
from constants import DATA_PATH
import pandas as pd

df = pd.read_csv(DATA_PATH / 'sentinel-1-processed.csv')

df

,VH-pol,VV-pol,SM1 (%),Crop Name,label
0,-19.3557,-13.54610,4.6,Fallow,Low
1,-17.1605,-10.46910,7.2,Pearl Millet,Low
2,-17.8729,-9.98927,6.4,Fallow,Low
3,-17.7906,-11.93460,9.6,Lemon,Low
4,-17.3646,-10.73270,3.9,Fallow,Low
...,...,...,...,...,...
1811,-16.7871,-11.17010,50.0,Wheat,Very High
1812,-17.2167,-12.15740,50.0,Wheat,Very High
1813,-17.7785,-10.37820,50.0,Tobacco,Very High
1814,-16.6607,-10.62890,50.0,Tobacco,Very High
